In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

# 데이터 불러오기

In [ ]:
os.chdir('/content/drive/Othercomputers/내 MacBook Air/MLDL/project/open')

In [ ]:
train = pd.read_csv('./train.csv')
val = pd.read_csv('./val.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
train_total = train.iloc[:, 1:]
val_total = val.iloc[:, 1:]
test_total = test.iloc[:, 1:]

In [ ]:
val_total_x = val_total.drop(['Class'], axis=1)
val_total_y = val_total.loc[:, 'Class']

In [ ]:
from sklearn.preprocessing import StandardScaler  # 표준화 
std = StandardScaler()

train_total_std = std.fit_transform(train_total) 
val_total_x_std = std.fit_transform(val_total_x) 
test_total_std = std.fit_transform(test_total) 

## EllipticEnvelope

#### 라벨링
* +1 이면 boundary 안에 들어온 값으로 정상 데이터
* -1 이면 outlier

정상데이터: 0 , 이상치: 1로 라벨링 수정

In [ ]:
def get_pred_label(model_pred):
  model_pred = np.where(model_pred==1, 0, model_pred)
  model_pred = np.where(model_pred==-1, 1, model_pred)
  return model_pred

### EE Model Fit

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report
EE_model = EllipticEnvelope(support_fraction = 0.994, contamination = 0.00112, random_state = 42)

EE_model.fit(train_total_std)

EllipticEnvelope(contamination=0.00112, random_state=42, support_fraction=0.994)

### Predict

In [ ]:
val_EE_pred = EE_model.predict(val_total_x_std)
val_EE_pred = get_pred_label(val_EE_pred)
val_EE_score = f1_score(val_total_y, val_EE_pred, average='macro')
print(f'Validation F1 Score : [{val_EE_score}]')
print(classification_report(val_total_y, val_EE_pred))

Validation F1 Score : [0.9236496787663914]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.86      0.83      0.85        30

    accuracy                           1.00     28462
   macro avg       0.93      0.92      0.92     28462
weighted avg       1.00      1.00      1.00     28462



## AutoEncoder

In [ ]:
train_total_std.shape

(113842, 30)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
# Sequential

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers
# Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU

### Model

In [ ]:
input_dim = train_total_std.shape[1]
output_dim = input_dim
encoding_dim = 128

input_layer = layers.Input(shape=(input_dim, ))

encoder1 = layers.Dense(encoding_dim/2, name='encoder1')(input_layer)
encoder1 = layers.BatchNormalization()(encoder1)
encoder1 = layers.LeakyReLU()(encoder1)

encoder2 = layers.Dense(encoding_dim, name='encoder2')(encoder1)
encoder2 = layers.BatchNormalization()(encoder2)
encoder2 = layers.LeakyReLU()(encoder2)

decoder1 = layers.Dense(encoding_dim/2, name='decoder1')(encoder2)
decoder1 = layers.BatchNormalization()(decoder1)
decoder1 = layers.LeakyReLU()(decoder1)

decoder2 = layers.Dense(output_dim, name='decoder2')(decoder1)

autoencoder = Model(input_layer, decoder2)
autoencoder.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

In [ ]:
print("====== 모델 전체 구조 ======")
print(autoencoder.summary())

====== 모델 전체 구조 ======
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 encoder1 (Dense)            (None, 64)                1984      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64)                0         
                                                                 
 encoder2 (Dense)            (None, 128)               8320      
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                      

### Fit

In [ ]:
EPOCHS = 400
BATCH = 2**14
reduce_lr = ReduceLROnPlateau(monitor='loss', mode='max', factor=0.5, patience=10, min_lr=1e-8, verbose=True)
# es = EarlyStopping(monitor='loss', patience=5, verbose=1)

In [ ]:
hist = autoencoder.fit(train_total_std, train_total_std, 
                 epochs=EPOCHS,
                 batch_size=BATCH,
                 callbacks=[reduce_lr],
                 shuffle=True)

Epoch 1/400
7/7 [==============================] - 7s 12ms/step - loss: 0.7145 - lr: 0.0100
Epoch 2/400
7/7 [==============================] - 0s 10ms/step - loss: 0.2410 - lr: 0.0100
Epoch 3/400
7/7 [==============================] - 0s 10ms/step - loss: 0.1337 - lr: 0.0100
Epoch 4/400
7/7 [==============================] - 0s 11ms/step - loss: 0.0966 - lr: 0.0100
Epoch 5/400
7/7 [==============================] - 0s 10ms/step - loss: 0.0750 - lr: 0.0100
Epoch 6/400
7/7 [==============================] - 0s 8ms/step - loss: 0.0611 - lr: 0.0100
Epoch 7/400
7/7 [==============================] - 0s 11ms/step - loss: 0.0525 - lr: 0.0100
Epoch 8/400
7/7 [==============================] - 0s 9ms/step - loss: 0.0462 - lr: 0.0100
Epoch 9/400
7/7 [==============================] - 0s 8ms/step - loss: 0.0412 - lr: 0.0100
Epoch 10/400
7/7 [==============================] - 0s 9ms/step - loss: 0.0373 - lr: 0.0100
Epoch 11/400
1/7 [===>..........................] - ETA: 0s - loss: 0.0355
Epoch 11

### Predict 예측

In [ ]:
val_y_pred = autoencoder.predict(val_total_x_std)

890/890 [==============================] - 2s 2ms/step


#### 예측값 코사인유사도 계산

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def cosine_similarity(true, pred):
  diff = []
  for i in range(len(true)):
    diff.append(cos_sim(true[i], pred[i]))
  diff = pd.Series(diff)
  return np.array(diff)

#### 예측 라벨링

In [ ]:
val_AE_pred = np.where(cosine_similarity(val_total_x_std, val_y_pred)>0.975, 0, 1)

In [ ]:
pd.DataFrame(val_total_y[val_total_y != val_AE_pred]).value_counts()

Class
0        88
1        29
dtype: int64

In [ ]:
val_AE_score = f1_score(val_total_y, val_AE_pred, average='macro')
print(f'Validation F1 Score : [{val_AE_score}]')
print(classification_report(val_total_y, val_AE_pred))

Validation F1 Score : [0.5073735224219078]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.01      0.03      0.02        30

    accuracy                           1.00     28462
   macro avg       0.51      0.52      0.51     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
preds.apply(mode,axis = 0)

In [ ]:
# (0,0)=>0  | (1,0)=>1  | (0,1)=>0  | (1,1)=>1
from collections import Counter

def mode (x) :
    cnt = Counter(x)
    mode = cnt.most_common(1)
    return mode[0][0]

In [ ]:
def get_ensemble_pred(test):
  # pred EE
  print('EE_model')
  test_EE_pred = EE_model.predict(test)
  test_EE_pred = get_pred_label(test_EE_pred)

  # AE
  print('AE_model')
  test_AE_pred = autoencoder.predict(test)
  test_AE_pred = np.where(cosine_similarity(test, test_AE_pred)>0.974, 0, 1)

  preds = pd.DataFrame(zip(test_EE_pred, test_AE_pred))
  preds.columns = ['pred_EE', 'pred_AE']
  
  return preds, preds.apply(mode,axis = 1)

In [ ]:
test_pred = get_ensemble_pred(test_total_std)[1]

EE_model
AE_model
4454/4454 [==============================] - 14s 3ms/step


In [ ]:
test_pred.value_counts()

0    142194
1       309
dtype: int64

In [ ]:
pd.DataFrame(val_total_y[val_total_y != val_EE_pred]).value_counts()

Class
1        5
0        4
dtype: int64

In [ ]:
val_score = f1_score(val_total_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_total_y, val_pred))

Validation F1 Score : [0.9236496787663914]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.86      0.83      0.85        30

    accuracy                           1.00     28462
   macro avg       0.93      0.92      0.92     28462
weighted avg       1.00      1.00      1.00     28462

